In [2]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
from config import keys
import requests
import json
import time
from datetime import datetime
from scipy.stats import linregress
key = keys["weather_key"]
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + key

#### Generating weather Data
The code below is used to generate 2000 random cities, then generate a list of non-duplicate cities, after that the code gathers weather data for the cities and exports that to a csv so you dont need to generate a new weather database file every time you use the code

Skip this section of code if you already have a csv


In [2]:
# Generate random cords
lats = np.random.uniform(high = 90,low = -90, size = 2000)
lngs = np.random.uniform(high = 180,low = -180, size = 2000)
cords = zip(lats,lngs)

In [3]:
# convert the cords into cities and then add them to a list without any duplicates
cities = []
for cord in cords : 
    city = citipy.nearest_city(cord[0],cord[1]).city_name
                               
    if city not in cities:
         cities.append(city)

In [ ]:
# Define where the city data will be listed
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Description": description,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print(f"City not found. Skipping... Error Code :{city_weather['cod']}")
        print(f"Api response = {city_weather}")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


# convert the dictionary to a dataframe 
city_data_df = pd.DataFrame(city_data)

# save the dataframe as a csv (so you dont need to load a new city list every time)
city_data_df.to_csv("weatherpy_database.csv",index_label="City_ID")
city_data_df

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | upernavik
Processing Record 2 of Set 1 | luderitz
Processing Record 3 of Set 1 | albany
Processing Record 4 of Set 1 | cape town
Processing Record 5 of Set 1 | punta arenas
Processing Record 6 of Set 1 | tuktoyaktuk
Processing Record 7 of Set 1 | oranjestad
Processing Record 8 of Set 1 | tiksi
Processing Record 9 of Set 1 | dikson
Processing Record 10 of Set 1 | saleaula
City not found. Skipping... Error Code :404
Api response = {'cod': '404', 'message': 'city not found'}
Processing Record 11 of Set 1 | wanning
Processing Record 12 of Set 1 | ribeira grande
Processing Record 13 of Set 1 | hermanus
Processing Record 14 of Set 1 | chuy
Processing Record 15 of Set 1 | port shepstone
Processing Record 16 of Set 1 | brumunddal
Processing Record 17 of Set 1 | atuona
Processing Record 18 of Set 1 | clive
Processing Record 19 of Set 1 | cukai
Processing Record 20 of Set 1 | arraial do cabo
Processing Rec

In [ ]:
# convert the dictionary to a dataframe 
city_data_df = pd.DataFrame(city_data)

# save the dataframe as a csv (so you dont need to load a new city list every time)
city_data_df.to_csv("weatherpy_database.csv",index_label="City_ID")


#### Analyzing data
Below the code will use the csv produced above to do analysis and visualizations

In [3]:
# Load the city_data_df from the csv and look at it 
city_data_df = pd.read_csv("weatherpy_database.csv",index_col = "City_ID")
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
City_ID,,,,,,,,,
0,Upernavik,72.7868,-56.1549,24.49,92,100,6.96,overcast clouds,2021-11-03 20:58:59
1,Luderitz,-26.6481,15.1594,58.96,61,3,15.50,clear sky,2021-11-03 20:58:36
2,Albany,42.6001,-73.9662,47.17,56,71,8.88,broken clouds,2021-11-03 21:00:02
3,Cape Town,-33.9258,18.4232,61.27,80,40,6.91,scattered clouds,2021-11-03 20:58:44
4,Punta Arenas,-53.1500,-70.9167,55.11,47,96,17.49,overcast clouds,2021-11-03 21:00:22
...,...,...,...,...,...,...,...,...,...
692,Tamandare,-8.7597,-35.1047,79.57,80,26,15.90,scattered clouds,2021-11-03 21:22:17
693,Hervey Bay,-25.2986,152.8535,67.01,80,45,9.44,scattered clouds,2021-11-03 21:22:18
694,Ambovombe,-25.1667,46.0833,71.01,85,2,1.81,clear sky,2021-11-03 21:22:18
